In [1]:
import os

from matplotlib import pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from tempfile import gettempdir

from l5kit.configs import load_config_data
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDatasetVectorized
from l5kit.planning.vectorized.closed_loop_model import VectorizedUnrollModel
from l5kit.planning.vectorized.open_loop_model import VectorizedModel
from l5kit.vectorization.vectorizer_builder import build_vectorizer

In [2]:
import os
os.environ["L5KIT_DATA_FOLDER"] = "/home/jeffrey_wu13579/prediction-dataset"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./config.yaml")

In [3]:
# ===== INIT DATASET
train_zarr = ChunkedDataset(dm.require(cfg["train_data_loader"]["key"])).open()

vectorizer = build_vectorizer(cfg, dm)
train_dataset = EgoDatasetVectorized(cfg, train_zarr, vectorizer)
print(train_dataset)

+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [4]:
URBAN_DRIVER = "Urban Driver"
OPEN_LOOP_PLANNER = "Open Loop Planner"

In [5]:
#model_name = URBAN_DRIVER
model_name = OPEN_LOOP_PLANNER

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

weights_scaling = [1.0, 1.0, 1.0]

_num_predicted_frames = cfg["model_params"]["future_num_frames"]
_num_predicted_params = len(weights_scaling)


if model_name == URBAN_DRIVER:
    # TODO
    print("Not implemented yet")
elif model_name == OPEN_LOOP_PLANNER:
    # with ego history
    model_path = "/home/jeffrey_wu13579/l5kit/examples/urban_driver/OL_HS.pt"
    model = torch.load(model_path)
else:
    raise ValueError(f"{model_name=} is invalid")

/home/jeffrey_wu13579/miniconda3/lib/python3.9/site-packages/torch/serialization.py:602: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [6]:
train_cfg = cfg["train_data_loader"]
train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"],
                              num_workers=train_cfg["num_workers"])
model = model.to(device)

/home/jeffrey_wu13579/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [7]:
weights_scaling = [1.0, 1.0, 1.0]

_num_predicted_frames = cfg["model_params"]["future_num_frames"]
_num_predicted_params = len(weights_scaling)

retrain_model = VectorizedModel(
    history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
    history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
    num_targets=_num_predicted_params * _num_predicted_frames,
    weights_scaling=weights_scaling,
    criterion=nn.L1Loss(reduction="none"),
    global_head_dropout=cfg["model_params"]["global_head_dropout"],
    disable_other_agents=cfg["model_params"]["disable_other_agents"],
    disable_map=cfg["model_params"]["disable_map"],
    disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"],
)

retrain_optimizer = optim.Adam(retrain_model.parameters(), lr=1e-3)

original_model = VectorizedModel(
    history_num_frames_ego=cfg["model_params"]["history_num_frames_ego"],
    history_num_frames_agents=cfg["model_params"]["history_num_frames_agents"],
    num_targets=_num_predicted_params * _num_predicted_frames,
    weights_scaling=weights_scaling,
    criterion=nn.L1Loss(reduction="none"),
    global_head_dropout=cfg["model_params"]["global_head_dropout"],
    disable_other_agents=cfg["model_params"]["disable_other_agents"],
    disable_map=cfg["model_params"]["disable_map"],
    disable_lane_boundaries=cfg["model_params"]["disable_lane_boundaries"],
)

original_optimizer = optim.Adam(original_model.parameters(), lr=1e-3)

EDITED MORE
EDITED MORE


In [8]:
state_dict = model.state_dict()
#print(state_dict.keys())
print(state_dict['global_head.output_embed.layers.0.weight'].shape)
print(state_dict['global_head.output_embed.layers.0.bias'].shape)
print(state_dict['global_head.output_embed.layers.1.weight'].shape)
print(state_dict['global_head.output_embed.layers.1.bias'].shape)
print(state_dict['global_head.output_embed.layers.2.weight'].shape)
print(state_dict['global_head.output_embed.layers.2.bias'].shape)

torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024, 1024])
torch.Size([1024])
torch.Size([36, 1024])
torch.Size([36])


In [9]:
# load state_dict into original model before clearing output embedding layers
original_model.load_state_dict(state_dict.copy())

<All keys matched successfully>

In [10]:
# reset output embedding to initialization state
nn.init.zeros_(state_dict['global_head.output_embed.layers.0.bias'])
nn.init.zeros_(state_dict['global_head.output_embed.layers.1.bias'])
nn.init.zeros_(state_dict['global_head.output_embed.layers.2.bias'])
nn.init.kaiming_normal_(state_dict['global_head.output_embed.layers.0.weight'], nonlinearity="relu")
nn.init.kaiming_normal_(state_dict['global_head.output_embed.layers.1.weight'], nonlinearity="relu")
nn.init.kaiming_normal_(state_dict['global_head.output_embed.layers.2.weight'], nonlinearity="relu")



tensor([[-0.0234,  0.0293,  0.0122,  ...,  0.0254, -0.0224, -0.0261],
        [ 0.0279, -0.0115,  0.0797,  ..., -0.0592, -0.0544, -0.0417],
        [ 0.0684, -0.0212, -0.0267,  ..., -0.0593,  0.0273,  0.0177],
        ...,
        [ 0.0461, -0.0243, -0.0640,  ..., -0.0226, -0.0347, -0.0568],
        [-0.0014,  0.0833,  0.0150,  ..., -0.0363,  0.0278,  0.0035],
        [ 0.0125,  0.0119,  0.0126,  ..., -0.0202,  0.0529,  0.0520]],
       device='cuda:0')

In [11]:
retrain_model.load_state_dict(state_dict)
#print(model.state_dict()['global_from_local.weight'].shape)

<All keys matched successfully>

In [12]:
print(original_model.state_dict()['global_head.output_embed.layers.2.bias'])
print(retrain_model.state_dict()['global_head.output_embed.layers.2.bias'])

tensor([ 1.8462e-02, -4.5514e-04, -2.9379e-03,  1.3890e-02, -8.1110e-04,
         2.1696e-03,  1.3675e-02, -1.4021e-03, -6.7859e-04,  1.3471e-02,
         1.6030e-05,  8.6390e-04,  1.3217e-02,  7.7693e-04,  1.8852e-03,
         1.3872e-02,  1.7191e-05, -9.5368e-05,  1.7190e-02,  8.6928e-05,
         1.1376e-03,  1.4521e-02,  3.8391e-04,  8.2661e-04,  1.7826e-02,
        -5.7734e-04,  2.4533e-03,  1.6243e-02,  8.1782e-04, -4.5500e-04,
         1.7954e-02,  1.5620e-03, -2.6035e-04,  1.6429e-02,  1.8072e-03,
        -9.0915e-04])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [13]:
retrain_model = retrain_model.to(device)
retrain_model.train()
torch.set_grad_enabled(True)

for param in retrain_model.parameters():
    param.requires_grad = False

In [14]:
for param in retrain_model.global_head.output_embed.parameters():
    param.requires_grad = True

In [15]:
false_count = 0
true_count = 0
for param in retrain_model.parameters():
    if param.requires_grad:
        true_count += 1
    else:
        false_count += 1
print(false_count, true_count)

24 6


In [16]:
tr_it = iter(train_dataloader)
#progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
progress_bar = tqdm(range(50))
losses_train = []

for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    # Forward pass
    data = {k: v.to(device) for k, v in data.items()}
    result = retrain_model(data)
    loss = result["loss"]
    # Backward pass
    retrain_optimizer.zero_grad()
    loss.backward()
    retrain_optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

loss: 0.0775330513715744 loss(avg): 0.10134446769952773: 100%|██████████| 50/50 [00:10<00:00,  4.84it/s] 


In [ ]:
print(progress_bar)

In [ ]:
print(cfg["train_params"]["max_num_steps"])

In [19]:
# seeing loss on the original model without cleared output embedding
original_model = original_model.to(device)
original_model.train()
torch.set_grad_enabled(True)
tr_it = iter(train_dataloader)
progress_bar = tqdm(range(50))
losses_train = []

for _ in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    # Forward pass
    data = {k: v.to(device) for k, v in data.items()}
    result = original_model(data)
    loss = result["loss"]
    # Backward pass
    original_optimizer.zero_grad()
    loss.backward()
    original_optimizer.step()

    losses_train.append(loss.item())
    progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")

loss: 0.13090012967586517 loss(avg): 0.13438864499330522: 100%|██████████| 50/50 [00:09<00:00,  5.29it/s]
